In [1]:
%matplotlib inline

import math
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import integrate
from scipy import special


sns.set(style="darkgrid")

# Homework 4
## Problem 1

A d-dimensional hypersphere is defined by the condition:
$$x_{o}^2 + x_{1}^2 + \ldots + x_{d-1}^2 \leq R^2$$
Its volume can be calculated analytically using the formula:
$$V_d (R) = \frac{\pi^{d/2} R^d}{\Gamma(\frac{d}{2} + 1)}$$

Write a program to determine the volume of unit hypershperes (R = 1) using a MonteCarlo integration method, for d = 2,3,4,5,6. You can use either the Hit-or-Miss or the Sample Mean method.

Produce the following using a "good" PRNG, like D1(A1r) or the Mersenne Twister (which one did you use?):

In [2]:
def lcg(a, c, m, x):
    return (a * x + c) % m

def lcgPeriod(a, c, m, seed):
    x = seed
    results = []
    while (len(results) != m) and (not x in results):
        results.append(x)
        x = lcg(a, c, m, x)
    print(" The period of an LCG with:\n a = {}, c = {}, m = {}, and seed = {}\n is {}".format(a, c, m, seed, len(results)))

def D1_A1r(seed):
    return lcg(2685821657736338717, 0, 18446744073709551616, seed)

In [3]:
def analyticalVolume(d, R):
    return (((math.pi**(d / 2.0)) * (R**d)) / (special.gamma((d / 2.0) + 1)))

### a)
Plots of the accuracy of your numerical results (with error bars) vs.  the number of points (N) generated.

### b)
What is the (approximate) accuracy vs N behavior of your results. Does this follow the expected N^(−1/2) behavior?

### c)
Compare the results you obtain with your good PRNG with a PRNG that has obvious problems, like LCG(5,3,32).  Describe how the "bad" PRNG’s results differ from your "good" results.

In [4]:
def bad_LCG(seed):
    return lcg(5,3, 32, seed)

## Problem 2
Evaluate the integral:
$$\int_{0}^{1} f(x)dx = \int_{0}^{1} \sqrt{1-x^2} dx = \frac{\pi}{4}$$
using importance sampling with
$$p(x) = A(1-x)$$ $$[x \leq 0]$$
and a ”good” PRNG, like D1(A1r) or the Mersenne Twister.

### a)
What is the appropriate value of A?

### b) 
What transformation can be used to allow us to generate random numbers x according to the PDF, p(x), in x ∈ [0,1] from uniformly distributed random numbers, r in [0,1]?

### c)
Make a plot of the deviation of your numerical result (including statistical uncertainties) from the analytic result vs. the number of points you generate, N , for N in the range 10^2 − 10^9.

### d)
What is the variance of f(x)/p(x) in the interval [0,1]? Compare this withthe variance of f(x) over the same interval.